In [1]:
import glob
import os
import subprocess
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mintpy import view, plot_network
from mintpy.objects import gnss, timeseries, ifgramStack
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, writefile, utils as ut
from scipy import signal
from skimage.transform import rescale
from urllib.request import urlretrieve
from matplotlib.backends.backend_pdf import PdfPages


In [37]:
ts_file_list= [r'C:\Users\bvarugu\Documents\Concord\Dsc\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\Lamont_Earthquake\ARIA_products\Dsc\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\Losangeles1\Dsc\mintpy\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\Losangeles2\Dsc\mintpy\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\SacDelta42\Dsc\mintpy\geo\corrected_ts_TRE_GNSS.h5',
               r'C:\Users\bvarugu\Documents\SacDelta42\frame470Dsc\mintpy\geo\corrected_ts_TRE_GNSS.h5']
geom_file_list= [r'C:\Users\bvarugu\Documents\Concord\Dsc\geometryGeo.h5',
               r'C:\Users\bvarugu\Documents\Lamont_Earthquake\ARIA_products\Dsc\geometryGeo.h5',
               r'C:\Users\bvarugu\Documents\Losangeles1\Dsc\mintpy\geometryGeo.h5',
               r'C:\Users\bvarugu\Documents\Losangeles2\Dsc\mintpy\geometryGeo.h5',
               r'C:\Users\bvarugu\Documents\SacDelta42\Dsc\mintpy\geo\geo_geometryRadar.h5',
               r'C:\Users\bvarugu\Documents\SacDelta42\frame470Dsc\mintpy\geo\geo_geometryRadar.h5'];

In [39]:
import mintpy.cli.generate_mask
for i in range(len(ts_file_list)):
    ts_file = ts_file_list[i];
    geom_file = geom_file_list[i];
    mask_file = os.path.join(os.path.dirname(ts_file),'maskTempCoh_0.9.h5');
    if 'geo_' in os.path.basename(geom_file):
        temp_coh_file = os.path.join(os.path.dirname(ts_file),'geo_temporalCoherence.h5');
    else:
        temp_coh_file = os.path.join(os.path.dirname(ts_file),'temporalCoherence.h5');
    if os.path.exists(temp_coh_file):
        iargs = [temp_coh_file, '-m', '0.9', '-o', mask_file]
        mintpy.cli.generate_mask.main(iargs);
    else:
        print('Temporal coherence file do not exist for',os.path.dirname(ts_file))

input temporalCoherence file: C:\Users\bvarugu\Documents\Concord\Dsc\temporalCoherence.h5
read C:\Users\bvarugu\Documents\Concord\Dsc\temporalCoherence.h5
create initial mask with the same size as the input file and all = 1
all pixels with nan value = 0
exclude pixels with value < 0.9
create HDF5 file: C:\Users\bvarugu\Documents\Concord\Dsc\maskTempCoh_0.9.h5 with w mode
create dataset /mask of bool       in size of (2190, 3844)         with compression=None
finished writing to C:\Users\bvarugu\Documents\Concord\Dsc\maskTempCoh_0.9.h5
time used: 00 mins 0.1 secs.
input temporalCoherence file: C:\Users\bvarugu\Documents\Lamont_Earthquake\ARIA_products\Dsc\temporalCoherence.h5
read C:\Users\bvarugu\Documents\Lamont_Earthquake\ARIA_products\Dsc\temporalCoherence.h5
create initial mask with the same size as the input file and all = 1
all pixels with nan value = 0
exclude pixels with value < 0.9
create HDF5 file: C:\Users\bvarugu\Documents\Lamont_Earthquake\ARIA_products\Dsc\maskTempCoh_0.9

In [27]:
if 'geo_' in os.path.basename(geom_file):
    print('geocoded')